In [ ]:
!pip install pandas python-docx

In [ ]:
# import libraries
import requests
from bs4 import BeautifulSoup as bs

from docx import Document

In [ ]:
product_names = []
processor_types = []
product_rams = []
product_storages = []
product_displays = []
product_features = []
product_prices = []

In [ ]:
# Initialize page number
page = 1

# Initialize a counter for the iterations
iteration = 0

while iteration < 5:
    # Generate URL for the current page
    url = f'https://www.startech.com.bd/laptop-notebook/laptop?filter_status=7&limit=90&page={page}'


    # Query the website and return the html to the variable ‘response’
    response = requests.get(url).text

    soup = bs(response, "html.parser")

    # Find all product details
    for product in soup.find_all('div', class_='p-item-details'):
      # If no products are found, we've reached the end of the pages
      if not product:
          break

      # Get product name and link
      product_name_link = product.find('h4', class_='p-item-name').find('a')
      if product_name_link:
          product_name = product_name_link.text
          product_names.append(product_name)
          print(f'Product Name: {product_name}')

      # Get product attributes
      product_attributes = product.find('div', class_='short-description').find_all('li')
      for attribute in product_attributes:
          attribute_text = attribute.text
          if 'Processor' in attribute_text:
              processor_type = attribute_text.replace("Processor: ", "").replace("\r", "")
              processor_types.append(processor_type)
              print(f'Processor: {processor_type}')

          elif 'RAM' in attribute_text and 'Storage' in attribute_text:
              ram_storage = attribute_text.replace("\r", "").split("Storage:")
              ram = ram_storage[0].replace("RAM: ", "")
              storage = ram_storage[1]
              product_rams.append(ram)
              product_storages.append(storage)
              print(f'RAM: {ram}')
              print(f'Storage: {storage}')

          elif 'Display' in attribute_text:
              product_display = attribute_text.replace("Display: ", "").replace("\r", "")
              product_displays.append(product_display)
              print(f'Display Size (Inch): {attribute_text.replace("Display: ", "")}')

          elif 'Features' in attribute_text:
              product_feature = attribute_text.replace("Features: ", "").replace("\r", "")
              product_features.append(product_feature)
              print(f'Features: {attribute_text.replace("Features: ", "")}')

      # Get product price
      product_price = product.find('div', class_='p-item-price').find('span')
      if product_price:
          price = product_price.text.replace('৳', '')
          product_prices.append(price)
          print(f'Product Price: {price}')

    # Increment the page number after each iteration
    page += 1

    # Increment the iteration counter
    iteration += 1

In [ ]:
# print(soup)

In [ ]:
product_names[:5]

In [ ]:
processor_types[:5]

In [ ]:
product_rams[:5]

In [ ]:
product_storages[:5]

In [ ]:
product_displays[:5]

In [ ]:
product_features[:5]

In [ ]:
product_prices[:5]

In [ ]:
product_prices_len = len(product_prices)
product_prices_len

# Export to Excel CSV

In [ ]:
import csv

In [ ]:
# Prepare data for CSV
rows = zip(product_names, processor_types, product_rams, product_storages, product_displays, product_features, product_prices)

# Write to CSV
with open('StarTechBot_Laptop_Data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Product Name", "Processor Type", "RAM", "Storage", "Display", "Features" ,"Price"])  # writing headers
    for row in rows:
        writer.writerow(row)

In [ ]:
from google.colab import files

# Download the file to your computer
files.download('StarTechBot_Laptop_Data.csv')

In [ ]:
import pandas as pd
StarTechBot_Laptop_Data = pd.read_csv("StarTechBot_Laptop_Data.csv")

In [ ]:
# Remove commas from the "RAM" column
StarTechBot_Laptop_Data['RAM'] = StarTechBot_Laptop_Data['RAM'].str.replace(',', '')

In [ ]:
StarTechBot_Laptop_Data.shape

In [ ]:
StarTechBot_Laptop_Data.head()

In [ ]:
StarTechBot_Laptop_Data.tail()

# **Turning the CSV file to a word document**


In [ ]:
# Create a new Word document
doc = Document()

In [ ]:
# Initialize a question counter
question_counter = 1

In [ ]:
# Iterate over the rows of the DataFrame
for i, row in StarTechBot_Laptop_Data.iterrows():
    # Create the questions and answers
    q1 = f"Question {question_counter}: What are the details of {row['Product Name']}?"
    a1 = f"Answer: {row['Product Name']} has - Processor type: {row['Processor Type']}; RAM: {row['RAM']}; Storage: {row['Storage']}; Display: {row['Display']}; Features: {row['Features']}. It's price is {row['Price']} BDT."
    question_counter += 1

    q2 = f"Question {question_counter}: What is the price of {row['Product Name']}?"
    a2 = f"Answer: It's price is {row['Price']} BDT."
    question_counter += 1

    q3 = f"Question {question_counter}: What are the features of {row['Product Name']}?"
    a3 = a1
    question_counter += 1


    # Add the questions and answers to the Word document
    doc.add_paragraph(q1)
    doc.add_paragraph(a1)
    doc.add_paragraph(q2)
    doc.add_paragraph(a2)
    doc.add_paragraph(q3)
    doc.add_paragraph(a3)

# Save the Word document
doc.save('StarTechBot_Laptop_Data.docx')

In [ ]:
files.download('StarTechBot_Laptop_Data.docx')